In [1]:
import nltk
from nltk.corpus import sentiwordnet as swn
from pycorenlp import StanfordCoreNLP
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
from nltk.corpus import wordnet as wn
import requests
import sys, os
import numpy as np
import pandas as pd

Warming up PyWSD (takes ~10 secs)... took 6.866290807723999 secs.


In [2]:
nlp = StanfordCoreNLP('http://localhost:9000')
dependency_parser = nlp.annotate

In [6]:
breakdown = swn.senti_synset('yummy.a.01')
print(breakdown.pos_score())

0.75


In [4]:
wn.synsets('love')

[Synset('love.n.01'),
 Synset('love.n.02'),
 Synset('beloved.n.01'),
 Synset('love.n.04'),
 Synset('love.n.05'),
 Synset('sexual_love.n.02'),
 Synset('love.v.01'),
 Synset('love.v.02'),
 Synset('love.v.03'),
 Synset('sleep_together.v.01')]

In [44]:
positive_lexicon = []
negative_lexicon = []

def read_lexicon():
    global positive_lexicon;
    global negative_lexicon;
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'positive-words.txt'), 'r') as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
         
        positive_lexicon = file.readlines()
    
    with open(os.path.join(os.path.abspath('opinion-lexicon-English/') , 'negative-words.txt'), 'r', encoding = "ISO-8859-1") as file:
        line = file.readline();
        while ";" in line:
            line = file.readline();
        
        negative_lexicon = file.readlines()
        
    positive_lexicon = list(map(lambda word: word.rstrip("\n\r"), positive_lexicon))
    negative_lexicon = list(map(lambda word: word.rstrip("\n\r"), negative_lexicon))
    
        
read_lexicon()
op_set = positive_lexicon + negative_lexicon

In [45]:
print(len(positive_lexicon), len(negative_lexicon))

2006 4783


In [46]:
def pos_tag(sentence):
    result = dependency_parser(sentence, properties={"outputFormat": "json", "annotators": "pos"})['sentences'][0]['tokens']
    res = []
    for pos in result:
        res.append((pos['word'], pos['pos']))
    return res
    
# def pos_tag(sentence):
#     '''url = "http://localhost:9000"
#     request_params = {"annotators": "pos"}
#     r = requests.post(url, data=sentence, params=request_params, timeout=120)
#     try:
#         results = r.json()['sentences'][0]['tokens']
#         res = []
#         for pos in results:
#             res.append((pos['word'], pos['pos']))
#         return res
#     except Exception as e:
#         print(e)
#         return []
#     '''
#     return nltk.pos_tag(nltk.word_tokenize(sentence))

In [47]:
negation = [
    "afraid",
    "can't",
    "cannot",
    "deny",
    "mean",
    "negate",
    "negation",
    "negative",
    "neither",
    "never",
    "no",
    "non",
    "none",
    "nor",
    "not",
    "nothing",
    "refusal",
    "refuse",
    "reject",
    "rejection",
#     "don't",
#     "shouldn't",
#     "wouldn't",
#     "didn't",
#     "though",
#     "although",
#     "wasn't",
#     "isn't",
#     "but"
]

from nltk.corpus import sentiwordnet as swn
import string

def predict_lexicon(opinions, sent):
    endscore = 0
    for opinion in opinions:
        if opinion in positive_lexicon:
            endscore += 1
        elif opinion in negative_lexicon:
            endscore -= 1

        words = sent.split(' ')
        word_around = []
        for x in range(0, len(words)):
            if words[x] in string.punctuation:
                continue
            try:
                if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                    word_around.append(words[x])
                elif (words[x-1] == opinion):
                    word_around.append(words[x])
            except:
                pass
        for neg in negation:
            if neg in word_around:
#                 print(sent)
#                 print(opinion)
#                 print(word_around)
                endscore *= (-1)
                break

    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity

def predict_sentiwordnet(opinions, sent):
    endscore = 0
    for opinion in opinions:
#         if opinion in positive_lexicon:
#             endscore += 1
#         elif opinion in negative_lexicon:
#             endscore -= 1
#         break
        try:
            score = swn.senti_synset(opinion + '.a.1')
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)

            words = sent.split(' ')
            word_around = []
            for x in range(0, len(words)):
                if words[x] in string.punctuation:
                    continue
                try:
                    if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                        word_around.append(words[x])
                    elif (words[x-1] == opinion):
                        word_around.append(words[x])
                except:
                    pass
            for neg in negation:
                if neg in word_around:
                    endscore *= (-1)
                    break
        except:
            pass
    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity, endscore

def predict_sentiwordnet_lesk(opinions, sent):
    endscore = 0
    for opinion in opinions:
#         if opinion in positive_lexicon:
#             endscore += 1
#         elif opinion in negative_lexicon:
#             endscore -= 1
#         break
        try:
#             tagged_sentence = pos_tag(sent)
#             op_with_tag = ('','')
#             for word, tag in tagged_sentence:
#                 if opinion == word:
#                     op_with_tag = (word, tag)
#                     break
                    
#             pos = ''
#             if 'NN' in op_with_tag[1]:
#                 pos = 'n'
#             elif 'JJ' in op_with_tag[1]:
#                 pos = 'a'
#             elif 'VB' in op_with_tag[1]:
#                 pos = 'v'
#             elif 'RB' in op_with_tag[1]:
#                 pos = 'r'
#             else:
            pos = 'a'
            
            print(adapted_lesk(sent, opinion, pos).name())
            score = swn.senti_synset(adapted_lesk(sent, opinion, pos).name())
            if score.pos_score() > score.neg_score():
                endscore += score.pos_score()
            else:
                endscore += score.neg_score() * (-1)
                
            words = sent.split(' ')
            word_around = []
            for x in range(0, len(words)):
                if words[x] in string.punctuation:
                    continue
                try:
                    if (words[x+1] == opinion) or (words[x+2] == opinion) or (words[x+3] == opinion) or (words[x+4] == opinion):
                        word_around.append(words[x])
                    elif (words[x-1] == opinion):
                        word_around.append(words[x])
                except:
                    pass
            for neg in negation:
                if neg in word_around:
                    endscore *= (-1)
                    break
           
        except Exception as e:
            print(e)
            pass
            
    if endscore > 0:
        polarity = 'positive'
    else:
        polarity = 'negative'
           
    return polarity, endscore

In [48]:
def calculate(category, tipe):
    df = pd.read_csv('Results/Sentiwordnet/hasil_'+ tipe + category +'.csv')
    tp, tn, fp, fn = 0, 0, 0, 0
    p = 'positive'
    n = 'negative'
    for index in range(0, len(df)):
        label = df['label'][index]
        predict = df['predict'][index]
        
        if label == p and predict == p:
            tp += 1
        elif label == n and predict == n:
            tn += 1
        elif label == n and predict == p:
            fp += 1
        elif label == p and predict == n:
            fn += 1
    
    precision = round(tp / (tp + fp), 2)
    recall = round(tp / (tp + fn), 2)
    f1 = round(2 * ((precision * recall) / (precision + recall)), 2)
    accuracy = round((tp+tn)/(tp+tn+fp+fn), 2)
    
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    f = 2 * ((p * r) / (p + r))
    
    
    print(tp, '\t', tn, '\t', fp, '\t', fn, '\t', '\t', precision, '\t', recall, '\t', f1, '\t', '\t', accuracy)
    return tp, tn, fp, fn, p, r, f, accuracy

In [49]:
def preprocessing(sentence):
    return sentence.lower()

def calculate_f1(p, r):
    return round(2*((p*r)/(p+r)),2)

In [50]:
import numpy as np

def print_hasil(tipe):
    sf = pd.DataFrame(columns=['TP', 'TN', 'FP', 'FN', 'precision', 'recall', 'f1', 'accuracy'])
    print('tp', '\t', 'tn', '\t', 'fp', '\t', 'fn', '\t', '\t', 'prec', '\t', 'rec', '\t', 'f1', '\t', '\t', 'accuracy')
    precision = []
    recall = []
    f1 = []
    cat = ['AMBIENCE', 'FOOD', 'SERVICE', 'PRICES']
    for c in cat:
        a, b, c, d, e, f, g, h = calculate(c, tipe)
        precision.append(e)
        recall.append(f)
        f1.append(g)
        sf = sf.append({'TP':a, 'TN':b, 'FP':c, 'FN':d, 'precision':e, 'recall':f, 'f1':g, 'accuracy': h}, ignore_index=True)
    sf.to_excel("Results/Calculation/sentiment-"+ tipe +".xlsx")
    print('', '\t', '', '\t', '', '\t', '', '\t', '\t', round(np.mean(precision),2), '\t', round(np.mean(recall),2), '\t', calculate_f1(round(np.mean(precision),2), round(np.mean(recall),2)), '\t', '\t')

In [51]:
import pandas as pd
def run_lex(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict'])
    print(len(df))
    count = 0
    for index in range(0, len(df)):
        opinion = []
        if type(df['opinion'][index]) != float:
            opinion = df['opinion'][index].split('|')
            
        prediction = predict_lexicon(opinion, preprocessing(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction
           }, ignore_index=True)
        count += 1
    
    sf.to_csv("Results/Sentiwordnet/hasil_lex_"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_lex_"+ category +".xlsx")

In [52]:
import pandas as pd
def run(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict','score'])
    for index in range(0, len(df)):
        opinion = []
        if type(df['opinion'][index]) != float:
            opinion = df['opinion'][index].split('|')
            
        prediction, score = predict_sentiwordnet(opinion, preprocessing(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction,
            'score': score
           }, ignore_index=True)
    
    sf.to_csv("Results/Sentiwordnet/hasil_swn"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_swn"+ category +".xlsx")

In [53]:
import pandas as pd
def run_lesk(category):
    df = pd.read_csv('Results/'+ category +'.csv')
    sf = pd.DataFrame(columns = ['id','review', 'opinion', 'label', 'predict','score'])
    for index in range(0, len(df)):
        opinion = []
        if type(df['opinion'][index]) != float:
            opinion = df['opinion'][index].split('|')
            
        prediction, score = predict_sentiwordnet_lesk(opinion, preprocessing(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
            'review': df['review'][index],
            'opinion': df['opinion'][index],
            'label': df['label'][index],
            'predict': prediction,
            'score': score
           }, ignore_index=True)
    
    sf.to_csv("Results/Sentiwordnet/hasil_lesk_"+ category +".csv")
    sf.to_excel("Results/Sentiwordnet/hasil_lesk_"+ category +".xlsx")

In [54]:
run_lex('FOOD')
run_lex('AMBIENCE')
run_lex('SERVICE')
run_lex('PRICES')

177
107
122
80


In [55]:
run('FOOD')
run('AMBIENCE')
run('SERVICE')
run('PRICES')

In [56]:
run_lesk('FOOD')
run_lesk('AMBIENCE')
run_lesk('SERVICE')
run_lesk('PRICES')

lousy.s.02
many.a.01
good.s.12
perplex.v.01
good.s.21
sake.n.01
uncivil.a.01
special.s.04
'NoneType' object has no attribute 'name'
particular.s.01
innovative.s.02
innovative.s.02
extensive.s.01
purple.s.01
bang-up.s.01
good.s.12
amazing.s.02
great.s.03
great.s.03
heavenly.a.03
fresh.s.12
good.s.13
prepared.s.03
prepared.s.03
worth.s.02
tasty.a.01
great.s.03
great.s.03
good.s.21
good.s.21
good.s.21
'NoneType' object has no attribute 'name'
bland.s.01
bland.s.01
great.s.03
great.s.03
great.s.03
great.s.03
great.s.03
great.s.03
great.s.03
great.s.03
great.s.03
great.s.03
incredible.a.01
incredible.a.01
incredible.a.01
cold.s.13
cold.s.13
inert.s.03
inert.s.03
good.s.13
good.s.13
fancy.a.01
good.s.13
better.s.03
bang-up.s.01
good.s.12
amaze.v.01
good.s.12
bang-up.s.01
bang-up.s.01
delightful.s.01
unusual.s.03
delightful.s.01
great.s.03
great.s.03
fry.v.02
fry.v.02
japanese.a.01
japanese.a.01
japanese.a.01
japanese.a.01
good.s.13
prepared.s.03
good.s.13
great.s.01
delightful.s.01
delightfu

friendly.s.02
patient.a.01
prompt.s.02
friendly.s.02
fantastic.s.02
small.s.08
prompt.s.02
nice.s.03
helpful.a.01
prompt.s.02
great.s.03
unappreciative.s.01
unappreciative.s.01
professional.s.05
good.s.21
'NoneType' object has no attribute 'name'
fantastic.s.02
professional.s.05
lone.s.03
poor.s.06
great.s.03
noisy.s.02
delightful.s.01
bad.s.04
bad.s.04
bad.s.04
bad.s.04
impeccable.s.02
atrocious.s.03
excellent.s.01
excellent.s.01
proper.s.04
dainty.s.04
dainty.s.04
dainty.s.04
dainty.s.04
regretful.a.01
prompt.s.02
great.s.03
impeccable.s.02
unobtrusive.a.01
present.a.02
top.a.01
impeccable.s.02
impeccable.s.02
excellent.s.01
friendly.s.02
dense.s.04
dense.s.04
quick.s.06
nice.s.03
cheap.a.01
exceptional.s.03
excellent.s.01
friendly.s.02
great.s.03
atrocious.s.03
uncivil.a.01
uncivil.a.01
worth.s.02
nice.s.03
nice.s.03
nice.s.03
'NoneType' object has no attribute 'name'
heedful.a.01
great.s.03
great.s.03
friendly.s.02
friendly.s.02
good.s.21
special.s.04
heedful.a.01
dainty.s.04
small

In [57]:
print_hasil('lex_') # lexicon

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
57 	 11 	 7 	 32 	 	 0.89 	 0.64 	 0.74 	 	 0.64
98 	 31 	 5 	 43 	 	 0.95 	 0.7 	 0.81 	 	 0.73
62 	 32 	 8 	 20 	 	 0.89 	 0.76 	 0.82 	 	 0.77
29 	 33 	 4 	 14 	 	 0.88 	 0.67 	 0.76 	 	 0.78
 	  	  	  	 	 0.9 	 0.69 	 0.78 	 	


In [58]:
print_hasil('') # lexicon + sentiwordnet

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
59 	 13 	 5 	 34 	 	 0.92 	 0.63 	 0.75 	 	 0.65
66 	 25 	 11 	 75 	 	 0.86 	 0.47 	 0.61 	 	 0.51
43 	 27 	 11 	 39 	 	 0.8 	 0.52 	 0.63 	 	 0.58
21 	 20 	 18 	 23 	 	 0.54 	 0.48 	 0.51 	 	 0.5
 	  	  	  	 	 0.78 	 0.53 	 0.63 	 	


In [59]:
print_hasil('lesk_') # lexicon + sentiwordnet + wsd

tp 	 tn 	 fp 	 fn 	 	 prec 	 rec 	 f1 	 	 accuracy
50 	 11 	 7 	 39 	 	 0.88 	 0.56 	 0.68 	 	 0.57
87 	 24 	 12 	 54 	 	 0.88 	 0.62 	 0.73 	 	 0.63
52 	 27 	 13 	 30 	 	 0.8 	 0.63 	 0.7 	 	 0.65
15 	 25 	 12 	 28 	 	 0.56 	 0.35 	 0.43 	 	 0.5
 	  	  	  	 	 0.78 	 0.54 	 0.64 	 	
